# Overview of Finetuning Project

The purpose of this series of notebooks on Finetuning is to experiment with using aerial photos as a basis for predicting forest fire likelyhood based on the Modis datasets using Google Earth Engine Modis data and USDA/NAIP/DOQQ aerial photos - all from within the State of California




# Sampling Method

I used the Modis datasets to identify which regions in CA had been burned in forest fires in the period from 2018 to 2020.

I then collected images from these same regions from the period 2016 to end of 2017 and used the pre-burn images from both known fire and non-fire regions to make a resnet 18 binary model to predict Fire or NoFire for a given image.

The sampling index map is displayed below for both the FIre and NoFire cases showing the locations we sampled with aerial photos

## Fire Samples
<img src="CA Burn areas 2018 to 2021 sampled.png" width="800"/>

## NoFire Samples
<img src="CA No Burn areas 2018 to 2020 sampled.png" width="800"/>

## Notices and Disclaimers

Intel technologies may require enabled hardware, software or service activation.

No product or component can be absolutely secure. 

Your costs and results may vary. 

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. Other names and brands may be claimed as the property of others. 